In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data = np.load('data/npy/data_mel.npy')
labels = np.load('data/npy/labels_mel.npy')

In [20]:

# Daten in Tensoren konvertieren
data_tensor = torch.from_numpy(np.array(data)).float()

import torchvision.transforms as transforms

# Define the mean and standard deviation for normalization
mean = data_tensor.mean()
std = data_tensor.std()

# Create the transform
transform = transforms.Normalize(mean=mean, std=std)

# Reshape the data_tensor to have the shape (batch_size, channels, height, width)
data_tensor_for_normalisazion = data_tensor.view(-1, 1, 1, data_tensor.shape[1])
# Apply the transform to the data_tensor
normalized_data = transform(data_tensor_for_normalisazion)

labels_tensor = torch.from_numpy(np.array(labels))

# Teilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2)
device = "cpu"
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)


In [21]:
# save the data
torch.save(X_train, 'data/pt/X_train.pt')
torch.save(X_test, 'data/pt/X_test.pt')
torch.save(y_train, 'data/pt/y_train.pt')
torch.save(y_test, 'data/pt/y_test.pt')

In [9]:
# defining knn model
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        # Erstes lineares Layer
        self.fc1 = nn.Linear(88200, 22050)
        # ReLU Aktivierungsfunktion
        self.relu = nn.ReLU()
        # Dropout-Layer mit Wahrscheinlichkeit 0.2
        self.dropout = nn.Dropout(0.2)
        # Zweites lineares Layer
        self.fc2 = nn.Linear(22050, 5500)
        # ReLU Aktivierungsfunktion
        self.relu2 = nn.ReLU()
        # Drittes lineares Layer
        self.fc3 = nn.Linear(5500, 250)
        self.relu3 = nn.ReLU()
        # Dropout-Layer mit Wahrscheinlichkeit 0.2
        self.dropout2 = nn.Dropout(0.2)
        # Viertes lineares Layer
        self.fc4 = nn.Linear(250, 10)
        self.relu4 = nn.ReLU()
        # Fünftes lineares Layer
        self.fc5 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout2(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.softmax(x)
        return x



In [ ]:
# Modellinstanz
model = Model()

In [13]:
model = nn.Sequential(
    nn.Linear(88200, 22050),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(22050, 5500),
    nn.ReLU(),
    nn.Linear(5500, 250),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(250, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Softmax(dim=1)
)

In [14]:
device = "cpu"
model.to(device)

Sequential(
  (0): Linear(in_features=88200, out_features=22050, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=22050, out_features=5500, bias=True)
  (4): ReLU()
  (5): Linear(in_features=5500, out_features=250, bias=True)
  (6): ReLU()
  (7): Dropout(p=0.2, inplace=False)
  (8): Linear(in_features=250, out_features=10, bias=True)
  (9): ReLU()
  (10): Linear(in_features=10, out_features=1, bias=True)
  (11): Softmax(dim=1)
)

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = nn.BCEWithLogitsLoss()

In [23]:
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    output = output.squeeze()  # Remove dimensions of size 1
    loss = criterion(output, y_train.float())
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
